In [4]:
# TO DO:

#DROPNA IS NOT WORKING??
# markdown
# Add precipitation considerations
# Reset index near end.

In [17]:
import pandas as pd
import numpy as np
import os
import gmaps
import requests
import sys
from datetime import datetime
from pprint import pprint

sys.path.insert(0,'../')
from config import g_key

gmaps.configure(g_key)

In [13]:
weather_df = pd.read_csv(os.path.join("..","weather_database","WeatherPy_Database.csv"), index_col=0)
weather_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description
0,Castro,CL,-42.4721,-73.7732,57.38,90,90,11.50,light intensity shower rain
1,Ilulissat,GL,69.2167,-51.1000,15.82,35,100,12.66,overcast clouds
2,Dongsheng,CN,39.8161,109.9776,16.03,53,5,14.03,clear sky
3,Lufilufi,WS,-13.8745,-171.5986,78.58,76,47,19.82,light rain
4,Longyearbyen,SJ,78.2186,15.6401,12.04,57,0,12.66,clear sky
...,...,...,...,...,...,...,...,...,...
661,Pedasi,PA,7.5333,-80.0333,84.07,77,98,13.27,overcast clouds
662,Orlik,RU,52.5178,99.8250,-15.59,92,100,2.59,overcast clouds
663,Sinnar,SD,13.5691,33.5672,71.83,18,99,10.31,overcast clouds
664,Parrita,CR,9.5201,-84.3225,91.17,71,49,4.05,scattered clouds


In [14]:
# country decision engine:
    # DONE - ask user about temperature (already built)
    # ask user about rain (bin it, last step using exploratory analysis)
    # take results from df and generate count df of country values with more than 4 cities
        # add avg weather statistics if you're feeling spicy
    # Display list and force the user to select more broad city list of priorities aren't filled

In [15]:
#### BEGIN MIN/MAX TEMP PICKER #

### disallow unreasonable min/max values
min_temp_threshold = 100
max_temp_threshold = 0

#Input strings for repeatability.
min_temp_input_str = f"Please specify a minimum temperature no higher than {min_temp_threshold}: "
not_numeric_str = f"Chosen value is not a number."
threshold_str = f"Chosen value is outside specified threshold."
min_higher_than_max_str = f"Your max temperature is higher than your min temperature."

min_temp = input(min_temp_input_str)

# iterate on input until a valid value is given
while min_temp.isnumeric() == False or float(min_temp) > min_temp_threshold:
    if min_temp.isnumeric() == False:
        print(not_numeric_str)
    elif float(min_temp) > min_temp_threshold:
        print(threshold_str)
    min_temp = input(min_temp_input_str)
    
# Replace max threshold with the min value (if higher)
max_temp_threshold = max(float(min_temp),float(max_temp_threshold))
max_temp_input_str = f"Please specify a maximum temperature no lower than {int(max_temp_threshold)}: "

max_temp = input(max_temp_input_str)

# iterate on input until a valid value is given
while max_temp.isnumeric() == False or float(max_temp) < max_temp_threshold:
    if max_temp.isnumeric() == False:
        print(not_numeric_str)
    elif float(max_temp) < max_temp_threshold:
        print(threshold_str)
    max_temp = input(max_temp_input_str)
    
# Generate data frame from preferred temperatures and remove blanks
vacation_cities_df = weather_df.loc[(weather_df["Max Temp"] >= float(min_temp)) & (weather_df["Max Temp"] <= float(max_temp))].dropna()

# END MIN/MAX TEMP PICKER #

# BEGIN TABULATE RESULTS

# Import map to of ISO code to full country name and map to existing tables
country_code_map_df = pd.read_csv(os.path.join("..","weather_database","country_code_map.csv"))

country_counts_df = pd.DataFrame(data=vacation_cities_df.value_counts("Country"))
country_counts_df.columns = ["City Count"]

eligible_countries_df = pd.merge(left=country_counts_df,
                                 right=country_code_map_df,
                                 left_on = "Country",
                                 right_on = "Code",
                                 how="inner")

eligible_countries_df = eligible_countries_df.set_index("Code")

summary_weather_statistics_df = vacation_cities_df.groupby(["Country"]).mean()[["Max Temp"]]

eligible_countries_df = pd.merge(left=eligible_countries_df,
                                 right=summary_weather_statistics_df,
                                left_index=True,
                                right_index=True)

eligible_countries_df.columns = ["City Count", "Name", "Avg High Temp"]

# Reduce data set to countries capable of being mapped.
eligible_countries_df = eligible_countries_df.loc[(eligible_countries_df["City Count"] >= 4)]

print("\n------------------------------------------\n")
print("The trip planner requires at least four cities in one country to plan a full trip.")  

if len(eligible_countries_df) != 0:
    print("The following countries meet your criteria:")
    print(eligible_countries_df)
else: 
    print("No countries are eligible based on your criteria." + 
          "Please re-run the city picker in the above cell with broader selection criteria.")

Please specify a minimum temperature no higher than 100:  55
Please specify a maximum temperature no lower than 55:  80



------------------------------------------

The trip planner requires at least four cities in one country to plan a full trip.
The following countries meet your criteria:
    City Count                                   Name  Avg High Temp
US          21                          United States      66.071905
AU          21                              Australia      71.316667
ID          20                              Indonesia      75.812500
BR          17                                 Brazil      75.165882
IN          13                                  India      67.523846
ZA          12                           South Africa      65.775833
NZ          12                            New Zealand      67.432500
PE           9                                   Peru      74.164444
CL           8                                  Chile      65.280000
MX           8                                 Mexico      69.780000
AO           7                                 Angola      69.607143


In [26]:
gmaps_base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

vacation_cities_df["Hotel Name"] = np.nan

start_time = datetime.now()
log_file_name = os.path.join("logs", f"{start_time.timestamp()}.log")

# writing json calls to separate file in order to facilitate exploratory analysis
with open(log_file_name,mode="x") as log_file:

    log_start_msg = f"Commencing processing.  Start time: {start_time}\n"
    print(log_start_msg)
    log_file.write(log_start_msg)
    
    failed_lookup_count = 0
    
    for index, row in vacation_cities_df.iterrows():
                
        location = f'{vacation_cities_df.loc[index]["Lat"]},{vacation_cities_df.loc[index]["Long"]}'

        params = {'key':g_key,
                  'location':location,
                  'radius':5000,
                  'type':'logding'
                }

        city = row["City"]
        country = row["Country"]  

        try:
            gmaps_request = requests.get(url=gmaps_base_url, params=params)
            gmaps_json = gmaps_request.json()

            #results[0] just yields city name.
            hotel_name = gmaps_json['results'][1]['name']

            log_file.write(f"Identified {hotel_name} in {city}, {country}\n")        
            vacation_cities_df.loc[index, "Hotel Name"] = hotel_name

        except UnicodeEncodeError:
            log_file.write(f"Hotel name encoding error for {city}, {country}\n")
            failed_lookup_count += 1
        except:
            log_file.write(f"No hotels found for {city}, {country}\n")
            failed_lookup_count += 1
            
    log_end_msg = (f"\n------------------------------------------\n" +
                  f"Processing complete.  End time: {start_time}\n" +
                  f"{failed_lookup_count} cities unsuccessfully identified out of a total {len(vacation_cities_df)}.")
    print(log_end_msg)
    log_file.write(log_end_msg)
        
    print(f"Full logging details are available at {log_file_name}.")
        
vacation_cities_df = vacation_cities_df.dropna()

vacation_cities_df.reset_index(drop=True, inplace=True)
vacation_cities_df

Commencing processing.  Start time: 2022-01-01 17:11:55.408540


------------------------------------------
Processing complete.  End time: 2022-01-01 17:11:55.408540
13 cities unsuccessfully identified out of a total 293.
Full logging details are available at logs\1641085915.40854.log.


,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description,Hotel Name
0,Castro,CL,-42.4721,-73.7732,57.38,90,90,11.50,light intensity shower rain,Hotel Esmeralda
1,Lufilufi,WS,-13.8745,-171.5986,78.58,76,47,19.82,light rain,Leuaina Beach Resort and Spa
2,Bredasdorp,ZA,-34.5322,20.0403,61.29,82,96,8.05,overcast clouds,Bredasdorp Country Manor
3,Paris,US,33.6609,-95.5555,71.17,83,75,10.36,broken clouds,"Holiday Inn Express & Suites Paris, an IHG Hotel"
4,Lagoa,PT,39.0500,-27.9833,58.69,70,73,19.64,broken clouds,Casa Das Faias
...,...,...,...,...,...,...,...,...,...,...
275,Chimoio,MZ,-19.1164,33.4833,68.92,99,100,3.27,moderate rain,Faculty of Engineering
276,Bandipur,IN,11.6663,76.6286,62.74,96,92,4.07,overcast clouds,OYO 38131 Bandipur Khans Resort
277,Bhainsdehi,IN,21.6467,77.6325,55.76,75,8,2.33,clear sky,SBI ATM
278,Sinnar,SD,13.5691,33.5672,71.83,18,99,10.31,overcast clouds,Sennar University


In [28]:
vacation_cities_df.to_csv("WeatherPy_vacation.csv")

In [29]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Today's Weather</dt><dd>{Max Temp} and {Description}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in vacation_cities_df.iterrows()]

coordinates_df = vacation_cities_df[["Lat","Long"]]
coordinates_df

fig = gmaps.figure(center=(0.0, 0.0), 
                   zoom_level=1.5,
                   layout = {'width': '800px',
                             'height': '500px'})

markers = gmaps.marker_layer(locations=coordinates_df, info_box_content=hotel_info)
fig.add_layer(markers)

## Call the figure
fig

Figure(layout=FigureLayout(height='500px', width='800px'))